In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date,udf
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql import functions as F
import re

StatementMeta(SilverSpark, 14, 17, Finished, Available, Finished)

In [17]:
keywords={
    'ember.js', 'trello', 'symfony', 'laravel', 'javascript', 'ansible', 'zoom', 'monday.com', 'lisp', 'asp.net core', 'rust', 'groovy', 'terminal', 'asp.net', 'excel', 'objective-c', 'angular', 'C', 'bigquery', 'microstrategy', 'react.js', 'dplyr', 'nuxt.js', 'deno', 'crystal', 'unify', 'atlassian', 'sql server', 'python', 'ssis', 'sap', 'f#', 'ionic', 'vue.js', 'digitalocean', 'redis', 'ocaml', 'nosql', 'shell', 'word', 'gtx', 'clojure', 'scala', 'elasticsearch', 'powerpoint', 'phoenix', 'git', 'unix', 'homebrew', 'dax', 'php', 'vmware', 'elixir', 'suse', 'centos', 'wrike', 'puppet', 'dynamodb', 'couchbase', 'kali', 'smartsheet', 'hugging face', 'mlpack', 'dart', 'pulumi', 'rshiny', 'fastify', 'unreal', 'vue', 'visual basic', 'nuix', 'fastapi', 'kubernetes', 'matlab', 'bitbucket', 'perl', 'keras', 'nltk', 'mattermost', 'sas', 'sql', 'lua', 'capacitor', 'flow', 'airtable', 'apl', 'graphql', 'ubuntu', 'yarn', 'node.js', 'redhat', 'fortran', 'arch', 'notion', 'qt', 'hadoop', 'opencv', 'theano', 'matplotlib', 'db2', 'assembly', 'kafka', 'scikit-learn', 'cassandra', 'gatsby', 'wimi', 'debian', 'mariadb', 'pyspark', 'visualbasic', 'svn', 'no-sql', 'asana', 'mxnet', 'chainer', 'linode', 'workfront', 'gcp', 'sqlserver', 'jquery', 'redshift', 'delphi', 'twilio', 'julia', 'swift', 'unity', 'neo4j', 'spark', 'vba', 'openstack', 'java', 'docker', 'play framework', 'esquisse', 'rocketchat', 'xamarin', 'ssrs', 'haskell', 'next.js', 'R', 'confluence', 'asp.netcore', 'planner', 'qlik', 'terraform', 'bash', 'powershell', 'ggplot2', 'dlib', 'electron', 'ruby on rails', 'blazor', 'angular.js', 'golang', 'spreadsheet', 'ibm cloud', 'gdpr', 'react', 'tidyr', 'microsoft lists', 'mongo', 'ringcentral', 'wire', 'spring', 'flask', 'aws', 'npm', 'c#', 'html', 'css', 'clickup', 'datarobot', 'cobol', 'ovh', 'couchdb', 'shogun', 'splunk', 'sass', 'flutter', 't-sql', 'codecommit', 'symphony', 'chef', 'rubyon rails', 'mlr', 'microsoft teams', 'ruby', 'linux', 'dingtalk', 'pascal', 'msaccess', 'looker', 'gitlab', 'drupal', 'airflow', 'seaborn', 'wsl', 'firestore', 'cognos', 'c++', 'fedora', 'sheets', 'numpy', 'plotly', 'github', 'power bi', 'colocation', 'typescript', 'sharepoint', 'mysql', 'svelte', 'jira', 'heroku', 'postgresql', 'sqlite', 'azure', 'vb.net', 'webex', 'tableau', 'databricks', 'kotlin', 'erlang', 'node', 'pytorch', 'alteryx', 'django', 'solidity', 'mongodb', 'outlook', 'tidyverse', 'windows', 'ms access', 'snowflake', 'selenium', 'slack', 'powerbi', 'cordova', 'huggingface', 'watson', 'jupyter', 'aurora', 'firebase', 'visio', 'express', 'oracle', 'google chat', 'spss', 'macos', 'jenkins', 'tensorflow', 'pandas'}

StatementMeta(SilverSpark, 14, 18, Finished, Available, Finished)

In [18]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")



StatementMeta(SilverSpark, 14, 19, Finished, Available, Finished)

In [19]:
bronze_jobs_path = "abfss://bronze@datalakeyassin.dfs.core.windows.net/jobs/"
silver_jobs_path = "abfss://silver@datalakeyassin.dfs.core.windows.net/jobs/"
gold_jobs_path = "abfss://gold@datalakeyassin.dfs.core.windows.net/jobs/"

StatementMeta(SilverSpark, 14, 20, Finished, Available, Finished)

In [20]:
bronze_jobs_path = "abfss://bronze@datalakeyassin.dfs.core.windows.net/jobs/"
silver_jobs_path = "abfss://silver@datalakeyassin.dfs.core.windows.net/jobs/"


columns_map = {
    "data_jobs1.csv": ["Role", "Location", "Posted_Date", "Skills"],
    "data_jobs2.csv": ["job_title_short", "job_location", "job_posted_date", "job_skills"],
    "data_jobs3.csv": ["job_title", "full_address", "job_posted_date", "job_description_text"],
    "data_jobs4.csv": ["title", "location", "job_posted_date", "skills"],
    "data_jobs5.csv": ["title", "location", "posted_date", "description"]
}


unique_cols = ["job_title", "job_location", "job_posted_date", "job_skills"]

df_list = []
for file_name, cols_to_keep in columns_map.items():
    df = spark.read.option("header", True).csv(f"{bronze_jobs_path}{file_name}")
    df = df.select([col(c) for c in cols_to_keep])

    for old_col, new_col in zip(cols_to_keep, unique_cols):
        df = df.withColumnRenamed(old_col, new_col)
    df_list.append(df)

df_jobs = df_list[0]
for df in df_list[1:]:
    df_jobs = df_jobs.unionByName(df)


df_jobs = df_jobs.withColumn("job_posted_date", to_date(col("job_posted_date"), "yyyy-MM-dd"))

StatementMeta(SilverSpark, 14, 21, Finished, Available, Finished)

In [21]:
def extract_tools(text):
    found = []
    if text:
        for kw in keywords:
            if len(kw) <= 2:
                pattern = r'\b' + re.escape(kw) + r'\b'
                if re.search(pattern, text):
                    found.append(kw)
            else:
                pattern = r'\b' + re.escape(kw.lower()) + r'\b'
                if re.search(pattern, text.lower()):
                    found.append(kw)
    return found

extract_tools_spark_udf = udf(extract_tools, ArrayType(StringType()))

StatementMeta(SilverSpark, 14, 22, Finished, Available, Finished)

In [22]:
df_jobs = df_jobs.withColumn("tools", extract_tools_spark_udf("job_skills"))

StatementMeta(SilverSpark, 14, 23, Finished, Available, Finished)

In [23]:
it_job_titles = [
    "Data Scientist", "Data Analyst", "Business Analyst", "Data Engineer",
    "Machine Learning Engineer", "ML Ops Engineer", "NLP Engineer", "Computer Vision Engineer",
    "Software Engineer", "Backend Engineer", "Frontend Engineer", "Full Stack Engineer",
    "Mobile App Developer", "DevOps Engineer", "Cloud Engineer",
    "Python Developer", "Java Developer", "PHP Developer", "Angular Developer", "Vue.js Developer",
    "Database Developer", "ETL Developer"
]

def match_job(title):
    if not title:
        return None
    title_lower = title.lower()
    for job in it_job_titles:
        if job.lower() in title_lower:
            return job
    return title  # fallback: keep original

match_job_udf = udf(match_job, StringType())

df_jobs = df_jobs.withColumn("job_name", match_job_udf("job_title"))


StatementMeta(SilverSpark, 14, 24, Finished, Available, Finished)

In [24]:


df_jobs = df_jobs.withColumn("year", year("job_posted_date")) \
                      .withColumn("month", month("job_posted_date")) \
                      .withColumn("day", dayofmonth("job_posted_date")) \
                      .select("job_name", "job_location", "job_posted_date", "tools", "year", "month", "day")


StatementMeta(SilverSpark, 14, 25, Finished, Available, Finished)

In [25]:
df_jobs.show(5)

StatementMeta(SilverSpark, 14, 26, Finished, Available, Finished)

+--------------------+------------+---------------+--------------------+----+-----+---+
|            job_name|job_location|job_posted_date|               tools|year|month|day|
+--------------------+------------+---------------+--------------------+----+-----+---+
|      Data Scientist|   Tokyo, JP|     2025-07-25|          [excel, R]|2025|    7| 25|
|      Data Scientist|  London, UK|     2025-07-08|[python, azure, e...|2025|    7|  8|
|          BI Analyst|  Berlin, DE|     2025-06-10|[azure, tensorflo...|2025|    6| 10|
|      Data Scientist|  Berlin, DE|     2025-05-31|[sql, azure, aws,...|2025|    5| 31|
|Computer Vision E...| Toronto, CA|     2025-06-10|[aws, hadoop, doc...|2025|    6| 10|
+--------------------+------------+---------------+--------------------+----+-----+---+
only showing top 5 rows



In [26]:
df_github.write.mode("append").partitionBy("year", "month").parquet(silver_jobs_path)

StatementMeta(SilverSpark, 14, 27, Finished, Available, Finished)

In [27]:

staging_jobs = (
    df_jobs
    .withColumn("skill", F.explode("tools"))   
    .groupBy("job_name", "skill", "job_location", "year", "month", "day")
    .agg(
        F.count("*").alias("job_postings")  # number of postings for this job+skill+location+dat
    )
)

staging_jobs.show(10)


StatementMeta(SilverSpark, 14, 28, Finished, Available, Finished)

+--------------------+-------+-------------+----+-----+---+------------+
|            job_name|  skill| job_location|year|month|day|job_postings|
+--------------------+-------+-------------+----+-----+---+------------+
|AI Research Scien...|    sql|   London, UK|2025|    6| 12|           1|
|     ML Ops Engineer|    aws|Bengaluru, IN|2025|    5| 18|           1|
|AI Research Scien...|pytorch|   Sydney, AU|2025|    5|  8|           2|
|     ML Ops Engineer| pandas|   Sydney, AU|2025|    6|  2|           1|
|     ML Ops Engineer| hadoop|   London, UK|2025|    5| 26|           1|
|     ML Ops Engineer|  excel|   Mumbai, IN|2025|    7| 11|           1|
|        Data Analyst|  azure|Bengaluru, IN|2025|    7|  3|           1|
|AI Research Scien...|tableau|   Berlin, DE|2025|    7| 20|           1|
|      Data Scientist|pytorch|   Mumbai, IN|2025|    5|  7|           1|
|        NLP Engineer|pytorch| New York, US|2025|    7|  6|           1|
+--------------------+-------+-------------+----+--

In [28]:
staging_social.write.mode("append").parquet(gold_jobs_path)

StatementMeta(SilverSpark, 14, 29, Finished, Available, Finished)